In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler

In [2]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

### Using only values for FaceAPI without neutral gave the best result

In [3]:
dataset = pd.read_csv('dataset.csv')

In [4]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]

In [5]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling personality values to range 0 - 1

In [6]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

# Scaling input values to Gaussian distributions

In [7]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

# Training

In [8]:
svr = SVR(kernel='rbf')
mor = MultiOutputRegressor(svr)

mor.fit(data_train_qt, labels_train)
print('Training finished')

Training finished


In [9]:
predictions = mor.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''Values for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

Values for validation set:
MAE: 1.9661609449300905
MSE: 5.729109077363858
RMSE: 2.3935557393476046
R2:  0.004468396566861654


In [10]:
data_test = data_test.loc[labels_test['ANS_AROUSAL'] != 1]
data_test = data_test.loc[labels_test['ANS_AROUSAL'] != 5]
data_test = data_test.loc[labels_test['ANS_AROUSAL'] != 9]
labels_test = labels_test.loc[labels_test['ANS_AROUSAL'] != 1]
labels_test = labels_test.loc[labels_test['ANS_AROUSAL'] != 5]
labels_test = labels_test.loc[labels_test['ANS_AROUSAL'] != 9]

data_test = data_test.loc[labels_test['ANS_VALENCE'] != 1]
data_test = data_test.loc[labels_test['ANS_VALENCE'] != 5]
data_test = data_test.loc[labels_test['ANS_VALENCE'] != 9]
labels_test = labels_test.loc[labels_test['ANS_VALENCE'] != 1]
labels_test = labels_test.loc[labels_test['ANS_VALENCE'] != 5]
labels_test = labels_test.loc[labels_test['ANS_VALENCE'] != 9]

data_test_mms = mms.fit_transform(data_test)
data_test_mms = pd.DataFrame(data_test_mms, columns=data_test.columns)
data_test_qt = quantile_transformer.fit_transform(data_test_mms)
data_test_qt = pd.DataFrame(data_test_qt, columns=data_test_mms.columns)

predictions = mor.predict(data_test_qt)

mae, mse, r2 = get_score(labels_test, predictions)
print(f'''Values for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

Values for validation set:
MAE: 1.651673636136459
MSE: 4.094256512003739
RMSE: 2.0234269228226993
R2:  0.021201879755054887


# Using full dataset

In [10]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :-3]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [11]:
mms = MinMaxScaler()

data_train_mms = mms.fit_transform(data_train)
data_train_mms = pd.DataFrame(data_train_mms, columns=data_train.columns)
data_validation_mms = mms.fit_transform(data_validation)
data_validation_mms = pd.DataFrame(data_validation_mms, columns=data_validation.columns)

In [12]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train_mms)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train_mms.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation_mms)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation_mms.columns)

In [ ]:
svr = SVR(kernel='rbf')
mor = MultiOutputRegressor(svr)

mor.fit(data_train_qt, labels_train)
print('Training finished')

In [ ]:
predictions = mor.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation, predictions)
print(f'''Values for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')